## CrewAI Demo 1 - Create Agents to Research and Write an Article

This is my version of the notebook **Lesson 2 - Create Agents to Research and Write an Article** of the DeepLearningAI course [Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai). 

I've streamlined the code a bit, removing some non-essential bits. I've also changed the LLM model from OpenAI...

### 1 Preliminaries
#### Warning control

In [1]:
import warnings
warnings.filterwarnings('ignore')

#### Set up LangChain tracing

In [2]:
import os
# 1. Set up LangSmith environment variables (replace with your actual values)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "CrewAI demo project"  

#### Import from the crewAI libray

In [3]:
from crewai import Agent, Task, Crew

2025-04-09 08:49:39,338 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.31.1.


#### Set the LLM

Set those environment variables that your model and tools require.

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

### Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

#### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

#### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

#### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

#### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Future: The Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the rapid advancements in AI technology and its potential implications

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural language processing and chatbots
- AI-powered automation and robotics
- Ethical considerations in AI development

2. Key P

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Future: The Latest Trends and Key Players in Artificial Intelligence

## Introduction:
Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and interact with customers. The rapid advancements in AI technology have led to innovations in machine learning, natural language processing, and automation, reshaping the future of work and society as a whole. As AI continues to evolve, it is crucial to explore the latest trends and key players driving this technological revolution.

## Latest Trends in Artificial Intelligence:
In the realm of AI, machine learning and deep learning algorithms have emerged as powerful tools for data analysis and predictive modeling. These algorithms enable computers to learn from vast amounts of data and make decisions without human intervention. Additionally, natural language processing and chatbots have revolutionized customer service and communication, providing personalized experiences and improving efficiency.

AI-powered automation and robotics have also gained traction in various industries, streamlining processes and increasing productivity. However, as AI technologies become more prevalent, ethical considerations in AI development have come to the forefront. It is essential to address issues such as bias in AI algorithms, data privacy concerns, and the impact of automation on jobs and society.

## Key Players in the AI Industry:
Several key players have emerged as leaders in the AI industry, driving innovation and shaping the future of AI technology. Companies like Google DeepMind, IBM Watson, and Amazon Web Services (AWS) have made significant contributions to AI research and development. Microsoft Azure, OpenAI, and Tesla are also at the forefront of AI advancements, pushing the boundaries of what is possible with artificial intelligence.

## Noteworthy News in Artificial Intelligence:
Recent breakthroughs in AI research have paved the way for exciting applications in healthcare, finance, transportation, and other industries. Collaborations between tech giants and AI startups have led to innovative solutions and products that enhance the capabilities of AI technology. Regulatory developments in AI ethics and governance are also shaping the future of AI, ensuring that AI technologies are developed and deployed responsibly.

## Audience Analysis:
For tech enthusiasts, professionals in the IT industry, and business leaders exploring AI implementation, staying informed on the latest trends and developments in AI is essential. By understanding industry insights and ethical considerations in AI development, readers can navigate the complex landscape of AI technology more effectively. This blog post aims to provide valuable insights and resources for readers looking to deepen their knowledge of AI and its implications.

## Call to Action:
I encourage readers to stay informed on AI developments through reputable sources such as research reports, tech publications, podcasts, and webinars. Engaging in discussions on AI ethics and implications in their respective fields can help readers gain a deeper understanding of the societal impact of AI technology. Exploring AI-related courses or workshops can also further readers' knowledge and expertise in the field of artificial intelligence.

In conclusion, Artificial Intelligence is a dynamic and rapidly evolving field that holds immense potential for innovation and transformation. By exploring the latest trends, key players, and noteworthy news in AI, readers can gain valuable insights into the future of AI technology and its impact on society. Stay informed, engage in discussions, and continue to explore the possibilities of AI in shaping the future.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).